In [1]:
import requests
# re - модуль для работы с регулярными выражениями
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

# Задание №1

In [2]:
def crawler(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 101:
            return
    
        if url in visited:
            continue
        visited.add(url)
        
        try:
            #получаем контент страницы и декодируем его
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
        
        #парсинг страницы
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        # re.sub() для замены всех ' +' на ' '
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 101:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"
        # re.findall() - найти все вхождения
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawler(links_tree)

In [3]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawler([start])

In [4]:
index = {i:k for i, k in enumerate(texts)} 

In [5]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

In [6]:
path = 'C:\\Users\\User\\Downloads\\InfoSearch\\InfoSearch_1\\index.txt'  # путь указывается индивидуально
open(path).read()

'0: https://m.habr.com/ru/all/\n1: https://m.habr.com/ru/flows/marketing/\n2: https://m.habr.com/ru/company/skillfactory/profile/\n3: https://m.habr.com/ru/users/cointegrated/\n4: https://account.habr.com/info/agreement\n5: https://m.habr.com/ru/feedback/\n6: https://m.habr.com/ru/post/540210/comments/\n7: https://m.habr.com/ru/news/t/543604/comments/\n8: https://m.habr.com/ru/company/selectel/profile/\n9: https://m.habr.com/ru/users/\n10: https://m.habr.com/ru/hubs/\n11: https://m.habr.com/ru/users/ruvds/\n12: https://m.habr.com/ru/users/val9670/\n13: https://m.habr.com/ru/post/543686/\n14: https://m.habr.com/ru/docs/companies/corpblogs/\n15: https://m.habr.com/ru/post/540210/\n16: https://m.habr.com/ru/company/otus/profile/\n17: https://m.habr.com/ru/users/Old_tutor/\n18: https://m.habr.com/ru/company/jugru/profile/\n19: https://m.habr.com/ru/company/kaspersky/blog/543690/comments/\n20: https://account.habr.com/info/confidential/\n21: https://m.habr.com/ru/company/cloudera/blog/54365

# Задание №2

In [40]:
from openpyxl import load_workbook
import pandas as pd
import os, pymorphy2

In [41]:
# pymystem3.Mystem — морфологический анализатор русского языка с поддержкой снятия морфологической неоднозначности, 
# разработанный компанией «Яндекс». Программа работает на основе словаря и способна формировать морфологические гипотезы 
# о незнакомых словах
lemmatizer = Mystem()

In [42]:
words_all = {}
lemma_words = {}
lemma_word_forms = {}

In [43]:
def get_sparsed_list(word_list):
    # получить список всех словоформ
    morph, lst = pymorphy2.MorphAnalyzer(), []
    for word in word_list:
        lemma = ""
        for x in morph.parse(word)[0].lexeme:
            lemma+=x.word + " "
        lst.append(f"{word} {lemma}\n")
    return lst

In [45]:
for url in texts:
    text = texts[url]
    # оставляем только слова
    text = re.sub(r'[^a-zA-Zа-яА-Я]+', ' ', text)
    
    # удаляем повторяющиеся
    words = " ".join(sorted(set(text.lower().split()), key=text.lower().split().index))
    words_all[url]=words
    words = words.split(' ')
    
    lemmas = lemmatizer.lemmatize(words_all[url])
    lemma_words[url] = ''.join(lemmas).strip()
    
    lemma_word_forms[url]=get_sparsed_list(words)    

In [46]:
for url in texts:
    with open('words.txt', 'w') as f:
        for i, score in enumerate(words):
            f.write(f"{score}\n")
    with open('lemma_words.txt', 'w') as f:
        for i, score in enumerate(lemma_word_forms[url]):
            f.write(f"{score}\n")

# Задание №3

In [47]:
docs = [index[k] for k in index]

In [48]:
# Создание инвертированного списка терминов (индекс)

reverse_index = {}

for url in lemma_words:
    # пройдемся по списку лемматизированных токенов
    for word in lemma_words[url].split(' '):
        # если слово не входит в reverse_index, то добавляем его туда
        if not word in reverse_index:
            reverse_index[word] = set([url])
        else:
            reverse_index[word].update([url])

In [49]:
# булев поиск по построенному индексу

logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    # слова, которые должны присутствовать на странице
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    # слова, которые не должны присутствовать на странице
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    # фразы, которые должны присутствовать на странице
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        # list.pop([i]) - Возвращает элемент [на указанной позиции], удаляя его из списка.
        query.pop(i)
        query.pop(i)
    
    # хотя бы одно из указанных тут слов должно присутствовать на странице
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [50]:
# random.choice() - возвращает случайный элемент из указанной последовательности
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [51]:
query = f'{x1} | {x2} & !{x3}'
print(query)

dataset | экономически & !zangasta


In [52]:
find(query)

{'https://m.habr.com/ru/company/itsoft/blog/543662/comments/',
 'https://m.habr.com/ru/company/skillfactory/blog/542862/',
 'https://m.habr.com/ru/post/543702/'}